# Summary of Notebook
    defined loyalty_flag based on maximum number of orders
    defined spender_flag based on total spent
    defined frequency_flag based on median_days_prior_ord
    dataframe has 30330084 rows
    exported as pickle
    saved as '04 Analysis','cust_loyalty_spender_frequency.pkl'

In [1]:
#importing libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import datetime as dt

In [2]:
#import orders_products_customers_combined
df_all = pd.read_pickle(r'C:\Users\Selena\Desktop\3-22 Instacart Basket Analysis\04 Analysis\all_busy_day_hour.pkl')
path = r'C:\Users\Selena\Desktop\3-22 Instacart Basket Analysis'
df_all = pd.read_pickle(os.path.join(path, '04 Analysis', 'all_busy_day_hour.pkl'))

In [3]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30330084 entries, 0 to 30330083
Data columns (total 23 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   order_id                int32         
 1   user_id                 int32         
 2   order_number            int16         
 3   day_of_week_ordered     int16         
 4   hour_of_day_ordered     int16         
 5   days_since_prior_order  float64       
 6   product_id              int32         
 7   add_to_cart_order       int16         
 8   reordered               int16         
 9   product_name            category      
 10  aisle_id                int16         
 11  department_id           int16         
 12  price                   float64       
 13  gender                  category      
 14  state                   category      
 15  age                     int16         
 16  date_joined             datetime64[ns]
 17  dependents              int16         
 18  

In [4]:
#learning the maximum number of orders for each customer
df_all['max_order'] = df_all.groupby(['user_id'])['order_number'].transform(np.max)

In [5]:
df_all.head()

,order_id,user_id,order_number,day_of_week_ordered,hour_of_day_ordered,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,state,age,date_joined,dependents,living_situation,income,price_range_loc,busiest_day,busiest_period_of_day,max_order
0,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Alabama,31,2019-02-17,3,married,40423,Mid-range product,Least busy,Average Orders,10
1,473747,1,3,3,12,21.0,196,1,1,Soda,...,Alabama,31,2019-02-17,3,married,40423,Mid-range product,Least busy,Most Orders,10
2,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Alabama,31,2019-02-17,3,married,40423,Mid-range product,Least busy,Average Orders,10
3,431534,1,5,4,15,28.0,196,1,1,Soda,...,Alabama,31,2019-02-17,3,married,40423,Mid-range product,Least busy,Most Orders,10
4,3367565,1,6,2,7,19.0,196,1,1,Soda,...,Alabama,31,2019-02-17,3,married,40423,Mid-range product,Regularly busy,Average Orders,10


In [6]:
#defining loyal customers based on the number of max orders
df_all.loc[df_all['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [7]:
#defining average customers based on the number of max orders
df_all.loc[(df_all['max_order'] <= 40) & (df_all['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [8]:
#defining new customers based on the number of max orders
df_all.loc[df_all['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [9]:
#seeing the number of customers in each category
df_all['loyalty_flag'].value_counts(dropna=False)

Regular customer    15082338
Loyal customer      10095885
New customer         5151861
Name: loyalty_flag, dtype: int64

In [10]:
df_all.head()

,order_id,user_id,order_number,day_of_week_ordered,hour_of_day_ordered,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,age,date_joined,dependents,living_situation,income,price_range_loc,busiest_day,busiest_period_of_day,max_order,loyalty_flag
0,2398795,1,2,3,7,15.0,196,1,1,Soda,...,31,2019-02-17,3,married,40423,Mid-range product,Least busy,Average Orders,10,New customer
1,473747,1,3,3,12,21.0,196,1,1,Soda,...,31,2019-02-17,3,married,40423,Mid-range product,Least busy,Most Orders,10,New customer
2,2254736,1,4,4,7,29.0,196,1,1,Soda,...,31,2019-02-17,3,married,40423,Mid-range product,Least busy,Average Orders,10,New customer
3,431534,1,5,4,15,28.0,196,1,1,Soda,...,31,2019-02-17,3,married,40423,Mid-range product,Least busy,Most Orders,10,New customer
4,3367565,1,6,2,7,19.0,196,1,1,Soda,...,31,2019-02-17,3,married,40423,Mid-range product,Regularly busy,Average Orders,10,New customer


In [11]:
#learning the total spent for each customer
df_all['total_spent'] = df_all.groupby(['user_id'])['price'].transform(np.sum)

In [12]:
df_all['total_spent'].describe()

count    3.033008e+07
mean     3.291085e+03
std      2.925917e+03
min      2.000000e+00
25%      1.106000e+03
50%      2.418000e+03
75%      4.663000e+03
max      2.579290e+04
Name: total_spent, dtype: float64

In [13]:
#defining high spending customers based on total spent
df_all.loc[df_all['total_spent'] >= 4663, 'spender_flag'] = 'High spender'

In [14]:
#defining low spending customers based on total spent
df_all.loc[df_all['total_spent'] < 1106, 'spender_flag'] = 'Low spender'

In [15]:
#defining average spending customers based on total spent
df_all.loc[(df_all['total_spent'] <4663) & (df_all['total_spent'] >=1106), 'spender_flag'] = 'Average spender'

In [16]:
#seeing number of customers in spending category
df_all['spender_flag'].value_counts(dropna=False)

Average spender    15164480
High spender        7583385
Low spender         7582219
Name: spender_flag, dtype: int64

In [17]:
df_all.head()

,order_id,user_id,order_number,day_of_week_ordered,hour_of_day_ordered,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,dependents,living_situation,income,price_range_loc,busiest_day,busiest_period_of_day,max_order,loyalty_flag,total_spent,spender_flag
0,2398795,1,2,3,7,15.0,196,1,1,Soda,...,3,married,40423,Mid-range product,Least busy,Average Orders,10,New customer,344.1,Low spender
1,473747,1,3,3,12,21.0,196,1,1,Soda,...,3,married,40423,Mid-range product,Least busy,Most Orders,10,New customer,344.1,Low spender
2,2254736,1,4,4,7,29.0,196,1,1,Soda,...,3,married,40423,Mid-range product,Least busy,Average Orders,10,New customer,344.1,Low spender
3,431534,1,5,4,15,28.0,196,1,1,Soda,...,3,married,40423,Mid-range product,Least busy,Most Orders,10,New customer,344.1,Low spender
4,3367565,1,6,2,7,19.0,196,1,1,Soda,...,3,married,40423,Mid-range product,Regularly busy,Average Orders,10,New customer,344.1,Low spender


In [18]:
#so creating a group based on the customer loyalty
df_all.groupby('loyalty_flag')

In [19]:
#to learn the type of spender based on customer loyalty
df_all.groupby('loyalty_flag').agg({'total_spent': ['mean']})

,total_spent
,mean
loyalty_flag,
Loyal customer,6054.524742
New customer,609.751861
Regular customer,2357.176130


In [20]:
#looking at the frequency of spending
df_all.groupby('user_id')

In [21]:
#median of days_since_prior_order
df_all['days_since_prior_order'].median()

8.0

In [22]:
#learning the median of days_since_prior_order for each customer
df_all['median_days_prior_ord'] = df_all.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [23]:
#checking for change
df_all[['user_id', 'loyalty_flag', 'spender_flag',  'median_days_prior_ord']].head()

,user_id,loyalty_flag,spender_flag,median_days_prior_ord
0,1,New customer,Low spender,20.5
1,1,New customer,Low spender,20.5
2,1,New customer,Low spender,20.5
3,1,New customer,Low spender,20.5
4,1,New customer,Low spender,20.5


In [24]:
#defining low frequency customers based on median_days_prior_ord
df_all.loc[df_all['median_days_prior_ord'] >= 20, 'frequency_flag'] = 'Non-frequent customer'

In [31]:
#defining average frequency customers based on median_days_prior_ord
df_all.loc[(df_all['median_days_prior_ord'] < 20) & (df_all['median_days_prior_ord'] > 10), 'frequency_flag'] = 'Regular customer'

In [32]:
#defining high frequency customers based on median_days_prior_ord
df_all.loc[df_all['median_days_prior_ord'] <= 10, 'frequency_flag'] = 'Frequent customer'

In [33]:
#seeing number of customers in frequency flag
df_all['frequency_flag'].value_counts(dropna=False)

Frequent customer    20675947
Regular customer      9654137
Name: frequency_flag, dtype: int64

In [34]:
df_all.head()

,order_id,user_id,order_number,day_of_week_ordered,hour_of_day_ordered,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,income,price_range_loc,busiest_day,busiest_period_of_day,max_order,loyalty_flag,total_spent,spender_flag,median_days_prior_ord,frequency_flag
0,2398795,1,2,3,7,15.0,196,1,1,Soda,...,40423,Mid-range product,Least busy,Average Orders,10,New customer,344.1,Low spender,20.5,Regular customer
1,473747,1,3,3,12,21.0,196,1,1,Soda,...,40423,Mid-range product,Least busy,Most Orders,10,New customer,344.1,Low spender,20.5,Regular customer
2,2254736,1,4,4,7,29.0,196,1,1,Soda,...,40423,Mid-range product,Least busy,Average Orders,10,New customer,344.1,Low spender,20.5,Regular customer
3,431534,1,5,4,15,28.0,196,1,1,Soda,...,40423,Mid-range product,Least busy,Most Orders,10,New customer,344.1,Low spender,20.5,Regular customer
4,3367565,1,6,2,7,19.0,196,1,1,Soda,...,40423,Mid-range product,Regularly busy,Average Orders,10,New customer,344.1,Low spender,20.5,Regular customer


In [43]:
output = pd.pivot_table(data=df_all, 
                        index=['loyalty_flag','spender_flag','frequency_flag'], 
                        values='total_spent',
                        aggfunc='mean')
output

total_spent
loyalty_flag     spender_flag    frequency_flag                
Loyal customer   Average spender Frequent customer  3318.009066
                                 Regular customer   3488.910553
                 High spender    Frequent customer  7714.231680
                                 Regular customer   5341.000000
                 Low spender     Frequent customer   844.908145
                                 Regular customer    598.800000
New customer     Average spender Frequent customer  1474.421008
                                 Regular customer   1452.211183
                 Low spender     Frequent customer   500.836722
                                 Regular customer    496.571716
Regular customer Average spender Frequent customer  2514.797030
                                 Regular customer   2151.699114
                 High spender    Frequent customer  5958.576014
                                 Regular customer   5670.619170
                 Low spender     Frequent customer   781.713281
                                 Regular customer    750.576786

In [44]:
#checking dataframe
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30330084 entries, 0 to 30330083
Data columns (total 29 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   order_id                int32         
 1   user_id                 int32         
 2   order_number            int16         
 3   day_of_week_ordered     int16         
 4   hour_of_day_ordered     int16         
 5   days_since_prior_order  float64       
 6   product_id              int32         
 7   add_to_cart_order       int16         
 8   reordered               int16         
 9   product_name            category      
 10  aisle_id                int16         
 11  department_id           int16         
 12  price                   float64       
 13  gender                  category      
 14  state                   category      
 15  age                     int16         
 16  date_joined             datetime64[ns]
 17  dependents              int16         
 18  

In [45]:
#exporting the new dataframe as a pickle
df_all.to_pickle(os.path.join(path, '04 Analysis','cust_loyalty_spender_frequency.pkl'))